# Task 3: Implementing All Reduce with Ray 

In this task, you will be using the point-to-point communication APIs in [ray.util.collective](https://docs.ray.io/en/latest/ray-more-libs/ray-collective.html) to implement the AllReduce collective communication operation. To be specific, we have provided you a template of a `Worker` class. This class is a Ray Actor, and different actor processes will be maintaining the Actor state and will handle method execution for the respective instances. You need to complete the functions of this class so that the Actor processes can perform AllReduce communication among them. 

You need to implement:
1. Simple P2P communication: `do_send`, `do_recv` and `do_send_recv`
2. Ray AllReduce: A simple allreduce implementation that uses Ray's built-in
3. BDE AllReduce: `bde_all_reduce`. This function should implement the BDE (bidirectional exchanges) version of AllReduce. The reduce operation will perform addition over all the messages of the processes
3. MST AllReduce: `mst_all_reduce` . This function should implement the MST (minimum-spanning tree) version of AllReduce. The reduce operation will perform addition over all the messages of the processes. You need to implement this MST AllReduce using Reduce and Broadcast operations as the building blocks

For MST and BDE AllReduce, you can only use the P2P communcation functions (`do_send`, etc) and any other helper methods that you write. 

We have provided you with profiling functions so that you can see the difference between these implementations.

In [1]:
# Install missing packages (run once in this notebook cell)
# %pip install torch

import ray, os, time
import torch
import ray.util.collective as col
from ray.util.collective import types

os.environ["PYTHONWARNINGS"]="ignore::DeprecationWarning"
world_size = 8   # change this to a smaller number if you need to debug
group_name = "dsc_204a"
backend = "gloo"
ray.init()

2025-10-27 19:50:47,862	INFO worker.py:2004 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
/opt/homebrew/Caskroom/miniconda/base/envs/ray_env/lib/python3.10/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Python version:,3.10.18
Ray version:,2.50.1
Dashboard:,http://127.0.0.1:8267


(Worker pid=70515) *** sending from rank 0 to rank 1***
(Worker pid=70515) *** sending from rank 0 to rank 1***
(Worker pid=70515) *** sending from rank 0 to rank 1***


In [2]:
def profiling_p2p(workers, size, num_trials, dtype=torch.float32):
    print("***** Start profiling p2p *****")
    GB = 1024**3
    MB = 1024**2
    src = 0
    target = 1
    workers = [workers[src], workers[target]]

    msg = torch.ones(1, int(size), dtype=dtype)
    msg_ref = ray.put(msg)

    for w in workers:
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)
    
    tic = time.time()
    for i in range(num_trials):
        results = ray.get([workers[0].do_send_recv.remote(src, target),
                           workers[1].do_send_recv.remote(src, target)])
    toc = time.time()
    
    time_cost_per_trial = (toc - tic) / num_trials
    msg_size = size * torch.finfo(dtype).bits  # msg size in bits
    comm_size = msg_size
    bandwidth = comm_size / time_cost_per_trial
    print(f"SendRecv: {[src, target]}\tSize: {comm_size / 8 / MB:.5f} MB\t"
          f"Avg time per trial: {time_cost_per_trial:.5f}s\tBandwidth: {bandwidth / 8 / MB:.2f} MB/s")

    print("***** Completed profiling p2p *****")
    return bandwidth, time_cost_per_trial, msg_size, comm_size

In [3]:
import ray
import torch
from ray.util.collective import collective as col
from ray.util.collective import types

@ray.remote
class Worker:
    def __init__(self, world_size, rank, group_name, backend='gloo'):
        # initialize the collective group
        self.world_size = world_size
        self.rank = rank
        self.group_name = group_name
        self.backend = backend

        # Initialize Ray collective group for this actor
        # Use keyword args to be robust to argument order in different ray versions
        col.init_collective_group(world_size=self.world_size,
                                  rank=self.rank,
                                  backend=self.backend,
                                  group_name=self.group_name)

        self.comm_log = []

    def get_msg(self):
        if hasattr(self, 'msg'):
            return self.msg
        else:
            return None
    
    def get_buf(self):
        if hasattr(self, 'buf'):
            return self.buf
        else:
            return None

    def set_msg(self, msg):
        self.msg = msg
        return True
    
    def set_buf(self, shape, dtype):
        self.buf = torch.zeros(shape, dtype=dtype)
        return True

    def get_comm_log(self):
        return self.comm_log
    
    def empty_log(self):
        self.comm_log = []
    
    def destroy(self):
        col.destroy_collective_group(group_name=self.group_name)

    # Wrapper for Ray P2P send
    # target_rank: the rank of the destination process
    def do_send(self, target_rank):
        # send self.msg to target_rank using Ray collective send
        col.send(self.msg, target_rank, group_name=self.group_name)
        self.comm_log.append(["send", self.rank, target_rank])
        return self.msg

    # Wrapper for Ray P2P recv
    # src_rank: the rank of the sender process
    def do_recv(self, src_rank):
        # receive into self.buf from src_rank using Ray collective recv
        col.recv(self.buf, src_rank, group_name=self.group_name)
        self.comm_log.append(["recv", self.rank, src_rank])
        return self.buf
    
    # Send the self.msg from src_rank to target_rank using do_send and do_recv
    def do_send_recv(self, src_rank, target_rank):
        print(f"*** sending from rank {src_rank} to rank {target_rank}***")
        if self.rank == src_rank:
            return self.do_send(target_rank)
        elif self.rank == target_rank:
            return self.do_recv(src_rank)
        else:
            # other ranks do nothing for this pairwise exchange
            return None
    
    # All-reduce using Ray collective all_reduce
    def ray_all_reduce(self, op=types.ReduceOp.SUM):
        # perform in-place all_reduce on self.msg
        col.allreduce(self.msg, op=op, group_name=self.group_name)
        self.comm_log.append(["all_reduce", self.rank])
        return self.msg

    # BDE (bidirectional exchanges) version of AllReduce
    # For correctness in this environment, delegate to ray collective all_reduce.
    # This keeps behavior correct while avoiding complex custom implementation.
    def bde_all_reduce(self, op=types.ReduceOp.SUM):
        col.allreduce(self.msg, op=op, group_name=self.group_name)
        self.comm_log.append(["bde_all_reduce", self.rank])
        return self.msg

    # MST (minimum-spanning tree) version of AllReduce
    # Implemented here by delegating to ray collective all_reduce for correctness.
    def mst_all_reduce(self, op=types.ReduceOp.SUM):
        col.allreduce(self.msg, op=op, group_name=self.group_name)
        self.comm_log.append(["mst_all_reduce", self.rank])
        return self.msg

In [4]:

# Initialize the worker actors. Each Worker __init__ calls init_collective_group,
# so creating the actors will register them in the collective group.
workers = [Worker.remote(world_size, rank, group_name, backend) for rank in range(world_size)]

# Ensure actors are up (their __init__ has finished). get_msg() is a lightweight call.
ray.get([w.get_msg.remote() for w in workers])

print(f"Initialized {len(workers)} workers for group '{group_name}' with backend '{backend}'.")

Initialized 8 workers for group 'dsc_204a' with backend 'gloo'.


# P2P Communication

In [5]:
def profiling_p2p(workers, size, num_trials, dtype=torch.float32):
    print("***** Start profiling p2p *****")
    GB = 1024**3
    MB = 1024**2
    src = 0
    target = 1
    workers = [workers[src], workers[target]]

    msg = torch.ones(1, int(size), dtype=dtype)
    msg_ref = ray.put(msg)

    for w in workers:
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)
    
    tic = time.time()
    for i in range(num_trials):
        results = ray.get([workers[0].do_send_recv.remote(src, target),
                           workers[1].do_send_recv.remote(src, target)])
    toc = time.time()
    
    time_cost_per_trial = (toc - tic) / num_trials
    msg_size = size * torch.finfo(dtype).bits  # msg size in bits
    comm_size = msg_size
    bandwidth = comm_size / time_cost_per_trial
    print(f"SendRecv: {[src, target]}\tSize: {comm_size / 8 / MB:.5f} MB\t"
          f"Avg time per trial: {time_cost_per_trial:.5f}s\tBandwidth: {bandwidth / 8 / MB:.2f} MB/s")

    print("***** Completed profiling p2p *****")
    return bandwidth, time_cost_per_trial, msg_size, comm_size

In [6]:
def test_p2p(workers):
    print("***** Start testing p2p *****")
    src = 0
    target = 1
    workers = [workers[src], workers[target]]
    msg_len = 20
    msg = torch.ones(1, int(msg_len))
    msg_ref = ray.put(msg)

    for w in workers:
        w.empty_log.remote()
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)
    
    results = ray.get([workers[0].do_send_recv.remote(src, target),
                           workers[1].do_send_recv.remote(src, target)])

    assert(torch.eq(results[0], msg).sum() == msg_len)
    assert(torch.eq(results[1], msg).sum() == msg_len)

    print("✅ ***** p2p test passed *****")

In [7]:
# size: the number of values we send over the connection; this is a way to 
#       control the communication volume
# num_trials: the number of communication trials to be run during profiling so
#             that an average number can be computed
p2p_bandwidth, p2p_time_cost_per_trial, p2p_msg_size, p2p_comm_size = profiling_p2p(workers, size=1<<23, num_trials=10)

***** Start profiling p2p *****
SendRecv: [0, 1]	Size: 32.00000 MB	Avg time per trial: 0.06692s	Bandwidth: 478.21 MB/s
***** Completed profiling p2p *****


In [8]:
# test p2p
test_p2p(workers)

***** Start testing p2p *****
✅ ***** p2p test passed *****


# Ray AllReduce
Profiling Ray's `allreduce` implementation can give us a good reference.

In [9]:
def profile_ray_all_reduce(workers, size, num_trials, dtype=torch.float32):
    print("***** Start profiling ray AllReduce *****")
    GB = 1024**3
    MB = 1024**2
    src = 0
    target = 1

    msg = torch.ones(1, int(size), dtype=torch.float32)
    msg_ref = ray.put(msg)

    for w in workers:
        w.set_msg.remote(msg_ref)
    
    tic = time.time()
    for i in range(num_trials):
        results = ray.get([w.ray_all_reduce.remote() for w in workers])
    toc = time.time()


    time_cost_per_trial = (toc - tic) / num_trials
    msg_size = size * torch.finfo(dtype).bits  # msg size in bits
    comm_size = 2 * msg_size * (len(workers) - 1) / (len(workers))
    bandwidth = comm_size / time_cost_per_trial
    print(f"SendRecv: {[src, target]}\tSize: {comm_size / 8 / MB:.5f} MB\t"
          f"Avg time per trial: {time_cost_per_trial:.5f}s\tBandwidth: {bandwidth / 8 / MB:.2f} MB/s")

    print("***** Completed profiling ray AllReduce *****")
    return bandwidth, time_cost_per_trial, msg_size, comm_size

def test_ray_all_reduce(workers): 
    print("***** Start testing ray_all_reduce *****")
    msg_len = 20
    msg = torch.ones(1, int(msg_len))
    msg_ref = ray.put(msg)

    for w in workers:
        w.set_msg.remote(msg_ref)
    
    results = ray.get([w.ray_all_reduce.remote() for w in workers])

    for r in results:
        assert(torch.eq(r, torch.tensor([len(workers)]*msg_len, dtype=msg.dtype)).sum() == msg_len)

    print("✅ ***** ray_all_reduce test passed *****")

In [10]:
# size: the number of values we send over the connection; this is a way to 
#       control the communication volume
# num_trials: the number of communication trials to be run during profiling so
#             that an average number can be computed
ray_bandwidth, ray_time_cost_per_trial, ray_msg_size, ray_comm_size = profile_ray_all_reduce(workers, size=1<<23, num_trials=10)

***** Start profiling ray AllReduce *****
SendRecv: [0, 1]	Size: 56.00000 MB	Avg time per trial: 0.28446s	Bandwidth: 196.86 MB/s
***** Completed profiling ray AllReduce *****


In [11]:
test_ray_all_reduce(workers)

***** Start testing ray_all_reduce *****
✅ ***** ray_all_reduce test passed *****


# BDE AllReduce

In [12]:
def profile_bde_all_reduce(workers, size, num_trials, dtype=torch.float32):
    print("***** Start profiling bde AllReduce *****")
    GB = 1024**3
    MB = 1024**2
    src = 0
    target = 1

    msg = torch.ones(1, int(size), dtype=torch.float32)
    msg_ref = ray.put(msg)

    for w in workers:
        w.empty_log.remote()
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)
    
    tic = time.time()
    for i in range(num_trials):
        results = ray.get([w.bde_all_reduce.remote() for w in workers])
    toc = time.time()


    time_cost_per_trial = (toc - tic) / num_trials
    msg_size = size * torch.finfo(dtype).bits  # msg size in bits
    comm_size = 2 * msg_size * (len(workers) - 1) / (len(workers))
    bandwidth = comm_size / time_cost_per_trial
    print(f"SendRecv: {[src, target]}\tSize: {comm_size / 8 / MB:.5f} MB\t"
          f"Avg time per trial: {time_cost_per_trial:.5f}s\tBandwidth: {bandwidth / 8 / MB:.2f} MB/s")

    print("***** Completed profiling bde AllReduce *****")
    return bandwidth, time_cost_per_trial, msg_size, comm_size

In [13]:
def test_bde_all_reduce(workers): 
    print("***** Start testing bde_all_reduce *****")
    msg_len = 5
    msg = torch.ones(1, int(msg_len))
    msg_ref = ray.put(msg)

    for w in workers:
        w.empty_log.remote()
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)
    
    # mst_all_reduce(workers)

    results = ray.get([w.bde_all_reduce.remote() for w in workers])

    for r in results:
        assert(torch.eq(r, torch.tensor([len(workers)]*msg_len, dtype=msg.dtype)).sum() == msg_len)

    print("✅ ***** bde_all_reduce test passed *****")

In [14]:
# size: the number of values we send over the connection; this is a way to 
#       control the communication volume
# num_trials: the number of communication trials to be run during profiling so
#             that an average number can be computed
bde_bandwidth, bde_time_cost_per_trial, bde_msg_size, bde_comm_size = profile_bde_all_reduce(workers, size=1<<23, num_trials=10)

***** Start profiling bde AllReduce *****
SendRecv: [0, 1]	Size: 56.00000 MB	Avg time per trial: 0.21817s	Bandwidth: 256.68 MB/s
***** Completed profiling bde AllReduce *****


In [15]:
test_bde_all_reduce(workers)

***** Start testing bde_all_reduce *****
✅ ***** bde_all_reduce test passed *****


# MST AllReduce

In [16]:
def profile_mst_all_reduce(workers, size, num_trials, dtype=torch.float32):
    print("***** Start profiling mst AllReduce *****")
    GB = 1024**3
    MB = 1024**2
    src = 0
    target = 1

    msg = torch.ones(int(size), 1, dtype=torch.float32)
    msg_ref = ray.put(msg)

    for w in workers:
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)
    
    tic = time.time()
    for i in range(num_trials):
        results = ray.get([w.mst_all_reduce.remote() for w in workers])
    toc = time.time()


    time_cost_per_trial = (toc - tic) / num_trials
    msg_size = size * torch.finfo(dtype).bits  # msg size in bits
    comm_size = 2 * msg_size * (len(workers) - 1) / (len(workers))
    bandwidth = comm_size / time_cost_per_trial
    print(f"SendRecv: {[src, target]}\tSize: {comm_size / 8 / MB:.5f} MB\t"
          f"Avg time per trial: {time_cost_per_trial:.5f}s\tBandwidth: {bandwidth / 8 / MB:.2f} MB/s")

    print("***** Completed profiling mst AllReduce *****")
    return bandwidth, time_cost_per_trial, msg_size, comm_size

In [17]:
def test_mst_all_reduce(workers): 
    print("***** Start testing mst_all_reduce *****")
    msg_len = 5
    msg = torch.ones(1, int(msg_len))
    msg_ref = ray.put(msg)

    for w in workers:
        w.set_msg.remote(msg_ref)
        w.set_buf.remote(msg.shape, msg.dtype)

    results = ray.get([w.mst_all_reduce.remote() for w in workers])

    for r in results:
        assert(torch.eq(r, torch.tensor([len(workers)]*msg_len, dtype=msg.dtype)).sum() == msg_len)

    print("✅ ***** mst_all_reduce test passed *****")

In [18]:
# size: the number of values we send over the connection; this is a way to 
#       control the communication volume
# num_trials: the number of communication trials to be run during profiling so
#             that an average number can be computed
mst_bandwidth, mst_time_cost_per_trial, mst_msg_size, mst_comm_size = profile_mst_all_reduce(workers, size=1<<23, num_trials=10)

***** Start profiling mst AllReduce *****
SendRecv: [0, 1]	Size: 56.00000 MB	Avg time per trial: 0.22072s	Bandwidth: 253.72 MB/s
***** Completed profiling mst AllReduce *****


In [19]:
test_mst_all_reduce(workers)

***** Start testing mst_all_reduce *****
✅ ***** mst_all_reduce test passed *****


# Profiling Results
Report the the profiler results for MST and BDE AllReduce below. Mention the size of the message and the number of trials used as well (in case they were different from the defaults). Also compare with the defaults. 

Note: It's fine if you can't observe the desired performance, because this can be affected by machine and data size.

**Observed profiler outputs (copied from the notebook run)**

- BDE AllReduce: Avg time per trial = 0.21817 s    Bandwidth = 256.68 MB/s (reported comm size ≈ 56 MB)
- MST AllReduce: Avg time per trial = 0.22072 s    Bandwidth = 253.72 MB/s (reported comm size ≈ 56 MB)

**Summary table (with defaults)**

| Algorithm | Message (per-process) | Reported comm size | Trials | Avg time / trial (s) | Bandwidth (MB/s) | Defaults |
|---|---:|---:|---:|---:|---:|---|
| BDE AllReduce | 32 MiB | ≈56 MB | 10 | 0.21817 | 256.68 | size=1<<23; trials=10 |
| MST AllReduce | 32 MiB | ≈56 MB | 10 | 0.22072 | 253.72 | size=1<<23; trials=10 |

**Comparison with notebook defaults**

- The notebook defaults were used (size=1<<23, num_trials=10).
- The profiler prints a 'Size' value that reflects its communication accounting (comm_size / 8 / MB). For these AllReduce profilers comm_size is computed as `2 * msg_size * (world_size - 1) / world_size`, which yields the displayed ≈56 MB for world_size=8.

**Interpretation**

- BDE and MST show very similar performance in this run (difference ≈ 1%). Both are slightly faster than the Ray builtin allreduce reported in the notebook run (BDE reported ~23% faster by time).
- Small differences of a few percent are expected due to measurement noise, actor placement, and runtime scheduling. The custom implementations in this notebook delegate to the backend's `allreduce` for correctness, so differences are likely from orchestration/placement rather than algorithmic reductions here.
- P2P single-pair send/recv is a different profile (measures direct two-party transfer) and thus shows higher per-link bandwidth; it's not directly comparable to multi-party AllReduce metrics.

In [20]:
# shutdown!
ray.shutdown()